In [18]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import sys

sys.path.append('./../../gustav/src')
from gustav import ncbi, nih, nlm, ebi
taxon = 9606

In [13]:
gene2lit = pd.read_parquet('../data/pubtator_title_or_abstract_in_any_gene2pubmed_paper_no_reviews.parquet')

In [68]:
ml = ncbi.pubmed('main', columns=['pubmed_id', 'year'], filters={'pubmed_id':set(gene2lit['pubmed_id'])})

In [74]:
gene2lit = pd.merge(gene2lit, ml, how='left')

In [75]:
gene2lit.to_csv('../data/pubtator_title_or_abstract_in_any_gene2pubmed_paper_no_reviews.csv', index=False)

In [14]:
gene_info = ncbi.gene_info(taxon, mode='unambiguous_ensembl')

C:\Users\richa\Documents\GitHub\gene_bot\notebooks\./../../gustav/src\gustav\utils.py:52: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_with_delimiter.drop(column, 1).reset_index(),
C:\Users\richa\Documents\GitHub\gene_bot\notebooks\./../../gustav/src\gustav\utils.py:63: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  joined = joined.drop('helper_index', 1)


In [22]:
uniprot = ebi.uniprot(dataset='uniprot_to_ncbi_gene', taxon_ncbi='vip')

In [26]:
gene_info = pd.merge(gene_info, uniprot, how='left')

In [58]:
df_list = []
n_entries = 60428
index_list = list(np.arange(0, n_entries, 2500)) + [n_entries]
for ii in range(len(index_list)-1):
    first_int = "{:05d}".format(index_list[ii]+1)
    last_int = "{:05d}".format(index_list[ii+1])
    filename = '../data/rcsb_pdb_custom_report_2d01832d49285ca19d69deec2ae5cbfe_'+first_int+'-' +last_int+'.csv'
    df = pd.read_csv(filename, header=1)
    df_list.append(df)
pdb_df = pd.concat(df_list).reset_index(drop=True)
pdb_df = pdb_df.dropna(subset=['PDB ID', 'Accession Code(s)'])
pdb_df = pdb_df[['PDB ID', 'Accession Code(s)']]
pdb_df.columns = ['pdb_id', 'protein_uniprot']

In [63]:
gene_info = pd.merge(gene_info, pdb_df, how='left')

In [65]:
gene_info.to_csv('../data/gene_info_221228.csv', index=False)